### _PROYECTO 2._ INTRODUCCIÓN AL ANÁLISIS DE DATOS

#### OBJETIVO

Poner en práctica las herramientas esenciales de Python para el análisis de datos a
partir de la clasificación y manejo de los mismos, mediante la creación y el uso de
archivos y estructuras de datos, implementación y creación de funciones, uso de
módulos importados y destructuring

#### DESCRIPCIÓN DEL CASO

Synergy Logistics es una empresa dedicada a la intermediación de servicios de
importación y exportación de diferentes productos. Actualmente la empresa
cuenta con una base de datos que refleja las rutas más importantes que opera
desde el año 2015, con su respectivo origen y destino, año, producto, modo de
transporte y valor total. Su propósito, es que a partir de estos datos se genere un
análisis que sirva de la base para la estructuración de su estrategia operativa.

#### CONSIGNA

La Dirección de Synergy Logistics ha solicitado al equipo operativo, realizar una
propuesta que permita enfocar las prioridades de la estrategia operativa 2021; para
ello, se plantea analizar la viabilidad de 3 opciones de enfoque: rutas de importación
y exportación, medio de transporte utilizado y valor total de importaciones y
exportaciones. Considerando que eres el data analyst del equipo, se te solicita que
realices un análisis de los siguientes puntos e identifiques cuál(es) de ellos es la
mejor opción para la empresa:

_Opción 1) Rutas de importación y exportación._

Synergy logistics está considerando la posibilidad de enfocar sus esfuerzos en las 10 rutas más
demandadas. Acorde a los flujos de importación y exportación, ¿cuáles son esas
10 rutas? ¿le conviene implementar esa estrategia? ¿porqué?

_Opción 2) Medio de transporte utilizado._

¿Cuáles son los 3 medios de transporte más importantes para Synergy logistics considerando el valor de las importaciones y exportaciones? ¿Cuál es medio de transporte que podrían
reducir?

_Opción 3) Valor total de importaciones y exportaciones._

Si Synergy Logistics quisiera enfocarse en los países que le generan el 80% del valor de las
exportaciones e importaciones ¿en qué grupo de países debería enfocar sus esfuerzos?


In [1]:
import pandas as pd
import numpy as np
import altair as alt
import operator
import copy
from collections import OrderedDict

#### Dataframe Principal


In [2]:
testDataframe=pd.read_csv('synergy_logistics_database.csv',parse_dates=[5])
testDataframe['month'] = testDataframe['date'].dt.month

### Opción 1) Rutas de importación y exportación.

Synergy logistics está considerando la posibilidad de enfocar sus esfuerzos en las 10 rutas más
demandadas. Acorde a los flujos de importación y exportación, ¿cuáles son esas
10 rutas? ¿le conviene implementar esa estrategia? ¿porqué?


#### Rutas más importantes globales 

Listar las 10 rutas con mas precencia 

In [3]:
# Listado de tipos de transporte
tipos_transporte=testDataframe['transport_mode'].unique()

num_rutas_totales=testDataframe.count()['register_id']

test_agrupar=testDataframe.groupby(['direction','origin','destination','transport_mode']).count().sort_values('register_id',ascending=False )['register_id']#.head(10)
# Transformacion de serie a DF
global_routes = pd.DataFrame(test_agrupar).reset_index()
global_routes["Porsentaje_rutas_global"] = round((global_routes['register_id']/num_rutas_totales)*100,3)
print(global_routes.head(10))

  direction       origin  destination transport_mode  register_id  \
0   Exports  South Korea      Vietnam            Sea          497   
1   Exports          USA  Netherlands            Sea          436   
2   Exports  Netherlands      Belgium           Road          374   
3   Exports        China       Mexico            Air          330   
4   Exports        Japan       Brazil            Sea          306   
5   Exports      Germany       France           Road          299   
6   Exports  South Korea        Japan            Sea          279   
7   Imports    Singapore     Thailand            Sea          273   
8   Exports    Australia    Singapore            Sea          273   
9   Exports       Canada       Mexico           Rail          261   

   Porsentaje_rutas_global  
0                    2.608  
1                    2.288  
2                    1.963  
3                    1.732  
4                    1.606  
5                    1.569  
6                    1.464  
7       

#### Divición de dataframe


In [4]:
# Agrupacion por rutas
agrupar_rutas=testDataframe.groupby(['direction','origin','destination','transport_mode']).count().sort_values('register_id',ascending=False )['register_id']
new_Dataframe_direc = pd.DataFrame(agrupar_rutas).reset_index()
# Separacion de rutas mas importantes  
# Importacion--> 
imp_data_head = new_Dataframe_direc.query('direction == "Imports"').sort_values('register_id',ascending=False ).head(5)
imp_data_head['ruta'] = imp_data_head['origin'] +'-->'+ imp_data_head['destination']
# Exportacion-->
exp_data_head = new_Dataframe_direc.query('direction == "Exports"').sort_values('register_id',ascending=False ).head(5)
exp_data_head['ruta'] = exp_data_head['origin'] +'-->'+ exp_data_head['destination']

####  Grafica de los datos de las  importaciónes (número-rutas)

In [5]:
print(imp_data_head.head())
bars = alt.Chart(imp_data_head).mark_circle().encode(
    y=alt.Y('ruta:O',title='Pais de origen ',scale=alt.Scale(zero=False)),
    x=alt.X('register_id', title='Número de rutas',scale=alt.Scale(zero=False, padding=2)),
    color=alt.Color("transport_mode", legend=alt.Legend(title="Tipo transporte"),scale=alt.Scale(scheme='dark2',type= 'sqrt'))
)

text = bars.mark_text(
    align='center',
    # baseline='middle',
    dx=15,
    size=13
).encode(
    text='register_id:Q'
)

(bars + text).properties(title='Número de rutas de importación por pais').interactive()


   direction     origin destination transport_mode  register_id  \
7    Imports  Singapore    Thailand            Sea          273   
12   Imports    Germany       China            Sea          233   
16   Imports      China       Japan            Air          210   
17   Imports      Japan      Mexico            Sea          206   
22   Imports   Malaysia    Thailand           Rail          195   

                    ruta  
7   Singapore-->Thailand  
12       Germany-->China  
16         China-->Japan  
17        Japan-->Mexico  
22   Malaysia-->Thailand  


alt.LayerChart(...)

####  Grafica de los datos de las  exportación (Conteo rutas)

In [6]:
print(exp_data_head)
bars = alt.Chart(exp_data_head).mark_circle().encode(
    y=alt.Y('ruta:O',title='Pais de origen ',scale=alt.Scale(zero=False)),
    x=alt.X('register_id', title='Número de rutas',scale=alt.Scale(zero=False, padding=2)),
    color=alt.Color("transport_mode", legend=alt.Legend(title="Tipo transporte"),scale=alt.Scale(scheme='dark2',type= 'sqrt')),   
)

text = bars.mark_text(
    align='center',
    dx=15,
    size=13
).encode(
    text='register_id:Q'
)

(bars + text).properties(title='Número de rutas de exportación por pais').interactive()


  direction       origin  destination transport_mode  register_id  \
0   Exports  South Korea      Vietnam            Sea          497   
1   Exports          USA  Netherlands            Sea          436   
2   Exports  Netherlands      Belgium           Road          374   
3   Exports        China       Mexico            Air          330   
4   Exports        Japan       Brazil            Sea          306   

                    ruta  
0  South Korea-->Vietnam  
1      USA-->Netherlands  
2  Netherlands-->Belgium  
3         China-->Mexico  
4         Japan-->Brazil  


alt.LayerChart(...)

In [7]:
# Agrupacion de datos extra
agrupar=testDataframe.groupby(['direction','origin','destination','product','transport_mode']).sum().sort_values('total_value',ascending=False )['total_value']#.head(10)
global_routess = pd.DataFrame(agrupar).reset_index()
global_routess['ruta']= global_routess['origin'] +'-->'+ global_routess['destination']

In [8]:
# Funcion para buscar el historial de una ruta
list_histo=[]
for index in range(5):
    first=global_routes.iloc[index]
    query=f'origin == "{first.origin}" and destination == "{first.destination}"  and transport_mode == "{first.transport_mode}" '
    first_route_hist = testDataframe.query(query).groupby(['year','origin','destination','transport_mode']).count()['register_id']
    first_route_hist = first_route_hist.to_frame().reset_index()
    list_histo.append(first_route_hist)
print(list_histo)

[   year       origin destination transport_mode  register_id
0  2015  South Korea     Vietnam            Sea          148
1  2018  South Korea     Vietnam            Sea           45
2  2019  South Korea     Vietnam            Sea          152
3  2020  South Korea     Vietnam            Sea          152,    year origin  destination transport_mode  register_id
0  2015    USA  Netherlands            Sea          106
1  2016    USA  Netherlands            Sea          113
2  2017    USA  Netherlands            Sea          142
3  2018    USA  Netherlands            Sea           75,    year       origin destination transport_mode  register_id
0  2015  Netherlands     Belgium           Road          166
1  2019  Netherlands     Belgium           Road           49
2  2020  Netherlands     Belgium           Road          159,    year origin destination transport_mode  register_id
0  2018  China      Mexico            Air           70
1  2019  China      Mexico            Air          146
2 

#### Historial de top 5 de rutas mas importantes

In [9]:
line = alt.Chart(list_histo[0]).mark_line().encode(
    x=alt.X('year:O' , title='Año'),
    y=alt.Y('register_id', title='Numero de rutas'),
    color=alt.Color("origin")
)

line1 = alt.Chart(list_histo[1]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

line2 = alt.Chart(list_histo[2]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

line3 = alt.Chart(list_histo[3]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

line4 = alt.Chart(list_histo[4]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

area= alt.Chart(list_histo[0]).mark_circle(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("origin:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area1= alt.Chart(list_histo[1]).mark_circle(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area2= alt.Chart(list_histo[2]).mark_circle(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area3= alt.Chart(list_histo[3]).mark_circle(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area4= alt.Chart(list_histo[4]).mark_circle(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

(line + line1 + line2 + line3  + line4 + area + area1 + area2 + area3 + area4 ).properties(width=500, title="Historial de top 5 rutas den 2020")



alt.LayerChart(...)

#### Grafica de ingresos por ruta

In [10]:
# Grafica general de ganancias por rutas
print(global_routess.head(5))
bars = alt.Chart(global_routess.head(15)).mark_point().encode(
    x=alt.X('ruta:O',title='Ruta ',scale=alt.Scale(zero=False),sort=alt.SortArray(['register_id'])),
    y=alt.X('total_value', title='Ingresos totales por ruta',scale=alt.Scale(zero=False, padding=3)),
    color=alt.Color("transport_mode", legend=alt.Legend(title="Tipo transporte"),scale=alt.Scale(scheme='dark2',type= 'sqrt')),  
    shape=alt.Shape('product',title='Tipo Producto'),
    tooltip=['direction','transport_mode', 'product', 'total_value', 'ruta']
)

text = bars.mark_text(
    align='center',
    # baseline='middle',
    dx=15,
    size=13
).encode(
    text='product',
)

(bars  ).properties(title='Ingresos por rutas-productos').interactive()


  direction       origin destination              product transport_mode  \
0   Exports        China      Mexico            Computers            Air   
1   Exports       Canada      Mexico                 Wood           Rail   
2   Exports          USA      Mexico      Aerospace Parts           Rail   
3   Exports  South Korea     Vietnam  Integrated circuits            Sea   
4   Imports    Singapore    Thailand  Integrated circuits            Sea   

   total_value                   ruta  
0   8438000000         China-->Mexico  
1   4508000000        Canada-->Mexico  
2   4370000000           USA-->Mexico  
3   4060007000  South Korea-->Vietnam  
4   4017000000   Singapore-->Thailand  


alt.Chart(...)

#### Grafica de numero de  relevancia de rutas

In [11]:
# Grafica general de numero rutas
print(global_routes.head(5))
bars = alt.Chart(global_routes.head(15)).mark_circle().encode(
    x=alt.X('origin:O',title='Pais de origen ',scale=alt.Scale(zero=False),sort=alt.SortArray(['register_id'])),
    y=alt.X('register_id', title='Numero de rutas',scale=alt.Scale(zero=False, padding=3)),
    color=alt.Color("transport_mode", legend=alt.Legend(title="Tipo transporte"),scale=alt.Scale(scheme='dark2',type= 'sqrt')),   
)

text = bars.mark_text(
    align='center',
    # baseline='middle',
    dx=15,
    size=13
).encode(
    text='register_id:Q'
)

(bars + text).properties(title='Numero de rutas por pais de origin')#.interactive()


  direction       origin  destination transport_mode  register_id  \
0   Exports  South Korea      Vietnam            Sea          497   
1   Exports          USA  Netherlands            Sea          436   
2   Exports  Netherlands      Belgium           Road          374   
3   Exports        China       Mexico            Air          330   
4   Exports        Japan       Brazil            Sea          306   

   Porsentaje_rutas_global  
0                    2.608  
1                    2.288  
2                    1.963  
3                    1.732  
4                    1.606  


alt.LayerChart(...)

#### Histograma rutas con mayores ingresos

In [12]:
print(testDataframe.columns.values)
test_histo=testDataframe.groupby(['origin', 'destination','year','product', 'transport_mode' ]).count()['register_id']#.sort_values(['total_value','date'],ascending=False )#.head(10)
global_routes_histo = pd.DataFrame(test_histo).reset_index()
# global_routes_histo['ruta']= global_routes_histo['origin'] +'-->'+ global_routes_histo['destination']
print(global_routes_histo)

['register_id' 'direction' 'origin' 'destination' 'year' 'date' 'product'
 'transport_mode' 'company_name' 'total_value' 'month']
             origin           destination  year          product  \
0         Australia                Brazil  2015  Coal Briquettes   
1         Australia                Brazil  2017  Coal Briquettes   
2         Australia                Brazil  2018  Coal Briquettes   
3         Australia                Brazil  2019  Coal Briquettes   
4         Australia                 Japan  2016             Meat   
..              ...                   ...   ...              ...   
516  United Kingdom                   USA  2017             Cars   
517  United Kingdom                   USA  2018             Cars   
518  United Kingdom                   USA  2019             Cars   
519  United Kingdom                   USA  2020             Cars   
520         Vietnam  United Arab Emirates  2020        Computers   

    transport_mode  register_id  
0              Sea 

#### _Opción 2) Medio de transporte utilizado._

¿Cuáles son los 3 medios de transporte más importantes para Synergy logistics considerando el valor de las importaciones y exportaciones? ¿Cuál es medio de transporte que podrían
reducir?


In [8]:
print(testDataframe.columns.values)
# Numero de rutas por tipo de transporte
medios_transporte_sta = testDataframe.groupby(['transport_mode']).count()['register_id'].sort_values(ascending=False).to_frame().reset_index()
print("Numero de rutas por tipo de transporte")
print(medios_transporte_sta,"\n")
# Ingresos por tipo de transporte 
medios_transporte_sta_sales = testDataframe.groupby(['transport_mode']).sum()['total_value'].sort_values(ascending=False).to_frame().reset_index()
print("Ingresos por tipo de transporte")
print(medios_transporte_sta_sales,"\n")
#  .mean()
medios_transporte_sta_sales_mean = testDataframe.groupby(['transport_mode']).mean()['total_value'].sort_values(ascending=False).to_frame().reset_index()
print("Promedi de ingreso por rutra y tipo de transporte ")
print(medios_transporte_sta_sales_mean,"\n")


['register_id' 'direction' 'origin' 'destination' 'year' 'date' 'product'
 'transport_mode' 'company_name' 'total_value']
Numero de rutas por tipo de transporte
  transport_mode  register_id
0            Sea        10688
1           Rail         3381
2           Road         2598
3            Air         2389 

Ingresos por tipo de transporte
  transport_mode   total_value
0            Sea  100530622000
1           Rail   43628043000
2            Air   38262147000
3           Road   33270486000 

Promedi de ingreso por rutra y tipo de transporte 
  transport_mode   total_value
0            Air  1.601597e+07
1           Rail  1.290389e+07
2           Road  1.280619e+07
3            Sea  9.405934e+06 



In [38]:
print(medios_transporte_sta)

grafic_count = alt.Chart(medios_transporte_sta).encode(
    theta=alt.Theta("register_id:Q", stack=True),
    radius=alt.Radius("register_id", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color="transport_mode:N",
)

c1 = grafic_count.mark_arc(innerRadius=20, stroke="#fff")

c2 = grafic_count.mark_text(radiusOffset=10,size=13).encode(text=alt.Text('register_id:Q'))
(c1 + c2).properties(title="Número de rutas por tipo de transporte")

  transport_mode  register_id
0            Sea        10688
1           Rail         3381
2           Road         2598
3            Air         2389


alt.LayerChart(...)

In [35]:
grafic_sales = alt.Chart(medios_transporte_sta_sales).encode(
    theta=alt.Theta("total_value:Q", stack=True),
    radius=alt.Radius("total_value", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color="transport_mode:N",
)

c3 = grafic_sales.mark_arc(innerRadius=20, stroke="#fff")

c4 = grafic_sales.mark_text(radiusOffset=10,size=13).encode(text=alt.Text('total_value:Q', format = ".3s"))

(c3 + c4).properties(title="Total de ingresos por ruta")

alt.LayerChart(...)

In [42]:

line = alt.Chart(medios_transporte_sta_sales_mean).mark_bar().encode(
    x=alt.X('transport_mode' , title='Pais de origen ',sort=alt.SortArray(['Porcentaje_apoyo_sales'])),
    y=alt.Y('total_value', title='Ingresos totales'),
    color=alt.Color("total_value", legend=alt.Legend(title="Ingresos"),scale=alt.Scale(scheme='goldgreen',type= 'sqrt')),   
)

area= alt.Chart(medios_transporte_sta_sales_mean).mark_area(opacity=0.3).encode(
    x=alt.X('transport_mode',sort=alt.SortArray(['total_value'])),
    y=alt.Y('total_value',stack=None,axis=alt.Axis(format=".2s") )
    # color="product"
).mark_text(align='center',dx=15,size=15).encode(text = alt.Text('total_value:Q', format = ".3s"))

(line + area ).properties(width=300, title="Ingresos promedio por tipo de ruta").interactive()



alt.LayerChart(...)

dataframes de estadisticas de precio por producto tipo envio y año

In [95]:
medios_transporte_sta_obj = testDataframe.groupby(['year','transport_mode']).count()['register_id'].sort_values(ascending=False)
print("Numero de rutas por tipo de transporte")
medios_transporte_sta_obj_DF = pd.DataFrame(medios_transporte_sta_obj).reset_index()
print(medios_transporte_sta_obj_DF,"\n")


# Ingresos por tipo de transporte 
medios_transporte_sta_sales_obj = testDataframe.groupby(['year','transport_mode']).sum()['total_value'].sort_values(ascending=True)
print("Ingresos por tipo de transporte")
medios_transporte_sta_sales_obj_DF = pd.DataFrame(medios_transporte_sta_sales_obj).reset_index()
# print(medios_transporte_sta_sales_obj_DF,"\n")
#  .mean()

medios_transporte_sta_sales_mean_obj = testDataframe.groupby(['year','transport_mode']).mean()['total_value'].sort_values(ascending=False)
medios_transporte_sta_sales_mean_obj_DF = pd.DataFrame(medios_transporte_sta_sales_mean_obj).reset_index()
print("Promedi de ingreso por rutra y tipo de transporte ")
#print(medios_transporte_sta_sales_mean_obj_DF,"\n")


Numero de rutas por tipo de transporte
    year transport_mode  register_id
0   2016            Sea         2004
1   2020            Sea         1902
2   2017            Sea         1874
3   2019            Sea         1813
4   2015            Sea         1619
5   2018            Sea         1476
6   2018           Rail          749
7   2019           Road          668
8   2017           Rail          660
9   2015           Road          609
10  2020           Rail          582
11  2018           Road          556
12  2015           Rail          542
13  2017            Air          536
14  2016           Rail          507
15  2015            Air          417
16  2018            Air          399
17  2020            Air          361
18  2019            Air          358
19  2016           Road          344
20  2019           Rail          341
21  2016            Air          318
22  2020           Road          310
23  2017           Road          111 

Ingresos por tipo de transporte
Pr

In [118]:
# grafica de ingresos por tipo de transporte 2015-2020
bars_sales = alt.Chart(medios_transporte_sta_sales_obj_DF).mark_line().encode(
    x=alt.X('year:O',title='Año',scale=alt.Scale(zero=False),sort='x'),
    y=alt.Y('total_value', title = "Ingresos",scale=alt.Scale(zero=False, padding=3),axis=alt.Axis(format = ".3s")),
    color=alt.Color("transport_mode", legend=alt.Legend(title="Tipo transporte"),scale=alt.Scale(scheme='dark2',type= 'sqrt')),
    tooltip=['year','transport_mode', 'total_value'],
)
# text = alt.Chart(medios_transporte_sta_sales_obj_DF).mark_text(align='center',dx=15,size=15).encode(text = alt.Text('total_value:Q', format = ".3s"))
text_sales = bars_sales.mark_text(
    align='center',
    baseline='top',
    dx=15,
    size=13 
).encode(text = alt.Text('total_value:Q', format = ".3s"))

# Grafica de numero de rutas por tipo de transporte 2015-2020
bars_count = alt.Chart(medios_transporte_sta_obj_DF).mark_line().encode(
    x=alt.X('year:O',title='Año ',scale=alt.Scale(zero=False),sort='x'),
    y=alt.Y('register_id', title = "Numero de rutas",scale=alt.Scale(zero=False, padding=3),axis=alt.Axis(format = ".3s")),
    color=alt.Color("transport_mode", legend=alt.Legend(title="Tipo transporte"),scale=alt.Scale(scheme='dark2',type= 'sqrt')),
    tooltip=['year','transport_mode', 'register_id'],
)

text_count = bars_count.mark_text(
    align='center',
    baseline='top',
    dx=15,
    size=13 
).encode(text = alt.Text('register_id:Q', format = ".3s"))

(bars_sales + text_sales).properties(width=600,title='Historial de numero de rutas por tipo de transporte').interactive() |(bars_count + text_count).properties(width=600,title='Historial de ingresos por tipo de transporte').interactive()

alt.HConcatChart(...)

#### Historial de las ventas por mes y por cada año segun el tipo de transporte

In [204]:
historic_mes=testDataframe.groupby(['year', 'month', 'transport_mode']).count()['register_id'].reset_index()
print(historic_mes.columns.values)

base = alt.Chart(historic_mes, width=60, height=alt.Step(8)).mark_line().encode(
    x=alt.Y("month:O", axis=None),
    y=alt.X("register_id", title=None,),
    color=alt.Color(
        "transport_mode:N", title="settings", legend=alt.Legend(orient="bottom", titleOrient="left")
    ),
    tooltip=['year' 'month' 'transport_mode' 'register_id'],
    row=alt.Row("year:N", title="Año", header=alt.Header(labelAngle=0)),
    column=alt.Column("transport_mode:N", title="Tipo de Transporte"),
)

(base).properties(width=200,title='Historial por mes-año para el tipo de transporte') 



['year' 'month' 'transport_mode' 'register_id']


ValueError: yearmonthtransport_moderegister_id encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)

#### Set de Países para mostrar en el mapa

Número de rutas en las que están involucrados los países ya sea exportación o importación


In [5]:
precencia_paices=testDataframe.loc[:,['origin','destination']]

precencia_paices= pd.unique(precencia_paices.values.ravel())

precencia_paices_dic = { country : 0 for country in precencia_paices }

count_precencia_orig=testDataframe['origin']

count_precencia_des=testDataframe['destination']

precencia_paices = pd.concat([count_precencia_orig,count_precencia_des ]).value_counts()

print(precencia_paices)


China                   3775
USA                     3386
Mexico                  2974
Germany                 2843
Japan                   2774
France                  1981
South Korea             1710
Canada                  1507
Netherlands             1495
Belgium                 1491
United Kingdom          1460
Brazil                  1418
Singapore               1123
Spain                   1110
Russia                  1103
Italy                   1070
Thailand                1032
Australia                965
India                    745
United Arab Emirates     635
Vietnam                  515
Austria                  503
Switzerland              443
Argentina                393
Malaysia                 288
New Zealand              249
Belorussia               219
Rusia                    204
Poland                   177
Ireland                  162
Philippines              114
Slovakia                  94
Turkey                    80
Peru                      44
Croatia       

### _Opción 3) Valor total de importaciones y exportaciones._

Si Synergy Logistics quisiera enfocarse en los países que le generan el 80% del valor de las
exportaciones e importaciones ¿en qué grupo de países debería enfocar sus esfuerzos?


#### EDA Exportacion

In [79]:
# Separar todo el df en la parte de expor e impo 
df_expo = testDataframe[(testDataframe.direction == "Exports")]

# Cantidad de exportaciones de todo el dataset
grupo_expor_count = df_expo.groupby(['direction']).count()['register_id']
print(grupo_expor_count)

# Cantidad de ingresos de exportaciones de todo el dataset
grupo_expor_sum = df_expo.groupby(['direction']).sum()['total_value']
# print(grupo_expor_sum)

# Paices que mas generan con exportacion (cantidad rutas)
grupo_expor_country = df_expo.groupby(['origin']).count()['register_id'].sort_values(ascending=False).to_frame().reset_index()
grupo_expor_country["Porcentaje_apoyo"] = round((grupo_expor_country.register_id/grupo_expor_count[0])*100,3)
# print(grupo_expor_country)

# ingreso por paices  exportacion 
grupo_expor_sales = df_expo.groupby(['origin']).sum()['total_value'].sort_values(ascending=False).to_frame().reset_index()
grupo_expor_sales["Porcentaje_apoyo_sales"] = round((grupo_expor_sales.total_value/grupo_expor_sum[0])*100,3)
grupo_expor_sales["Promedio"] = grupo_expor_sales['Porcentaje_apoyo_sales'].cumsum()

grupo_expor_sales=grupo_expor_sales[grupo_expor_sales['Promedio'] < 81]

# print(grupo_expor_sales.head(10))

direction
Exports    15408
Name: register_id, dtype: int64


####  Grafica de los datos de las  exportaciónes (Ingresos)

In [175]:

print(grupo_expor_sales.head())
base = alt.Chart(grupo_expor_sales.head(10)).encode(
    theta=alt.Theta("Porcentaje_apoyo_sales:Q", stack=True),
    radius=alt.Radius("Porcentaje_apoyo_sales", scale=alt.Scale(type="sqrt", zero=True, rangeMin=30)),
    color=alt.Color("Porcentaje_apoyo_sales:N", legend=alt.Legend(title='% de rutas')),   
)

c1 = base.mark_arc(innerRadius=20, stroke="#ffF")

c2 = base.mark_text(radiusOffset=10 ,size=14).encode(text="origin:N")

(c1 + c2).properties(width=500, title="Porcentaje ganancias de exportaciones por país")



NameError: name 'grupo_expor_sales' is not defined

In [177]:
line = alt.Chart(grupo_expor_sales.head(10)).mark_bar().encode(
    x=alt.X('origin' , title='Pais de origen ',sort=alt.SortArray(['Porcentaje_apoyo_sales'])),
    y=alt.Y('Porcentaje_apoyo_sales', title='Ingresos totales'),
    color=alt.Color("total_value", legend=alt.Legend(title="Ingresos"),scale=alt.Scale(scheme='goldgreen',type= 'sqrt')),   
)

area= alt.Chart(grupo_expor_sales.head(10)).mark_area(opacity=0.3).encode(
    x=alt.X('origin',sort=alt.SortArray(['Porcentaje_apoyo_sales'])),
    y=alt.Y('Porcentaje_apoyo_sales',stack=None,axis=alt.Axis(format=".2s") )
    # color="product"
).mark_text(align='left',dx=15,size=13).encode(text = alt.Text('total_value:Q', format = ".3s"))

(line + area ).properties(width=600, title="Ganancias por exportaciones").interactive()


NameError: name 'grupo_expor_sales' is not defined

####  Grafica de los datos de las  exportaciónes (Conteo rutas)

In [80]:
print(grupo_expor_country.head())
line = alt.Chart(grupo_expor_sales.head(10)).mark_point().encode(
    x=alt.X('origin' , title='Pais de origen ',sort=alt.SortArray(['Porcentaje_apoyo'])),
    y=alt.Y('Porcentaje_apoyo', title='Porsentaje de ingresosso totales'),
    # color=alt.Color("total_value"goldgreen )
    color=alt.Color("total_value", legend=alt.Legend(title="Ingresos"),scale=alt.Scale(scheme='goldgreen',type= 'sqrt')),   
)

area= alt.Chart(grupo_expor_country.head(10)).mark_area(opacity=0.3).encode(
    x=alt.X('origin',sort=alt.SortArray(['Porcentaje_apoyo'])),
    y=alt.Y('Porcentaje_apoyo',stack=None, )
    # color="product"
).mark_text(align='left',dx=15,size=13).encode(text='total_value:Q')

(line + area ).properties(width=500, title="Historial de top 5 rutas den 2020")

base = alt.Chart(grupo_expor_country.head(10)).encode(
    theta=alt.Theta("Porcentaje_apoyo:Q", stack=True),
    radius=alt.Radius("Porcentaje_apoyo", scale=alt.Scale(type="sqrt", zero=True, rangeMin=30)),
    color=alt.Color("Porcentaje_apoyo:N", legend=alt.Legend(title='% de rutas')),   
)

c1 = base.mark_arc(innerRadius=20, stroke="#ffF")

c2 = base.mark_text(radiusOffset=10 ,size=14).encode(text="origin:N")

(c1 + c2).properties(width=500, title="Porcentaje rutas de exportaciones por país")

        origin  register_id  Porcentaje_apoyo
0        China         1657            10.754
1          USA         1464             9.502
2      Germany         1295             8.405
3        Japan         1257             8.158
4  South Korea         1134             7.360


alt.LayerChart(...)

#### EDA Importación

In [13]:
# Separar todo el df en la parte de expor e impo 
df_import = testDataframe[(testDataframe.direction == "Imports")]

# Cantidad de exportaciones de todo el dataset
grupo_import_count = df_import.groupby(['direction']).count()['register_id']
# print(grupo_expor)

# Cantidad de ingresos de exportaciones de todo el dataset
grupo_import_sum = df_import.groupby(['direction']).sum()['total_value']
# print(grupo_import_sum)

# Paices que mas generan con exportacion (cantidad rutas)
grupo_import_country = df_import.groupby(['origin']).count()['register_id'].sort_values(ascending=False).to_frame().reset_index()
grupo_import_country["Porcentaje_apoyo"] = round((grupo_import_country.register_id/grupo_import_count[0])*100,3)


# ingreso por paices  exportacion 
grupo_inport_sales = df_import.groupby(['origin']).sum()['total_value'].sort_values(ascending=False).to_frame().reset_index()
grupo_inport_sales["Porcentaje_apoyo_sales"] = round((grupo_inport_sales.total_value/grupo_import_sum[0])*100,3)
grupo_inport_sales["Promedio"] = grupo_inport_sales['Porcentaje_apoyo_sales'].cumsum()
# print(grupo_inport_sales.head(10))
prom_acumulado_bol = grupo_inport_sales['Promedio'] < 82
grupo_inport_sales=grupo_inport_sales[prom_acumulado_bol]
# print(grupo_inport_sales.head(10))

# left join total_values count coun porsentaje 
left_join = pd.merge(grupo_inport_sales.head(10), grupo_import_country, how='left', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True)
print(left_join)




      origin_x  total_value  Porcentaje_apoyo_sales  Promedio   origin_y  \
0        China  12233000000                  22.030    22.030      China   
1        Japan   8094000000                  14.576    36.606      Japan   
2          USA   5291000000                   9.529    46.135        USA   
3       Mexico   4621000000                   8.322    54.457    Germany   
4      Germany   4250000000                   7.654    62.111     Mexico   
5    Singapore   4017000000                   7.234    69.345  Singapore   
6  South Korea   3889000000                   7.004    76.349   Malaysia   

   register_id  Porcentaje_apoyo  
0          615            16.859  
1          445            12.198  
2          376            10.307  
3          369            10.115  
4          366            10.033  
5          273             7.484  
6          199             5.455  


####  Grafica de datos de las  importaciónes (ingresos)

In [97]:
line = alt.Chart(grupo_inport_sales.head(10)).mark_bar().encode(
    x=alt.X('origin' , title='Pais de origen ',sort=alt.SortArray(['Porcentaje_apoyo_sales'])),
    y=alt.Y('Porcentaje_apoyo_sales', title='Ingresos totales'),
    color=alt.Color("total_value", legend=alt.Legend(title="Ingresos"),scale=alt.Scale(scheme='goldgreen',type= 'sqrt')),   
)

area= alt.Chart(grupo_inport_sales.head(10)).mark_area(opacity=0.3).encode(
    x=alt.X('origin',sort=alt.SortArray(['Porcentaje_apoyo_sales'])),
    y=alt.Y('Porcentaje_apoyo_sales',stack=None,axis=alt.Axis(format=".2s") )
    # color="product"
).mark_text(align='left',dx=15,size=13).encode(text = alt.Text('total_value:Q', format = ".3s"))

(line + area ).properties(width=600, title="Ganancias por importaciónes").interactive()


alt.LayerChart(...)

In [92]:
print(grupo_inport_sales)
base = alt.Chart(grupo_inport_sales.head(10)).encode(
    theta=alt.Theta("Porcentaje_apoyo_sales:Q", stack=True),
    radius=alt.Radius("Porcentaje_apoyo_sales", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color="Porcentaje_apoyo_sales:N",
)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="origin:N")

( c1 + c2 ).properties(width=500, title="Top rutas con mayores ingresos en importación")

        origin  total_value  Porcentaje_apoyo_sales  Promedio
0        China  12233000000                  22.030    22.030
1        Japan   8094000000                  14.576    36.606
2          USA   5291000000                   9.529    46.135
3       Mexico   4621000000                   8.322    54.457
4      Germany   4250000000                   7.654    62.111
5    Singapore   4017000000                   7.234    69.345
6  South Korea   3889000000                   7.004    76.349


alt.LayerChart(...)

####  Grafica de los datos de las  importaciónes (Conteo rutas)

In [90]:
print(grupo_import_country.head())
line = alt.Chart(grupo_import_country.head(10)).mark_point().encode(
    x=alt.X('origin' , title='Pais de origen ',sort=alt.SortArray(['Porcentaje_apoyo'])),
    y=alt.Y('Porcentaje_apoyo', title='Porsentaje de ingresosso totales'),
    # color=alt.Color("total_value"goldgreen )
    color=alt.Color("total_value", legend=alt.Legend(title="Ingresos"),scale=alt.Scale(scheme='goldgreen',type= 'sqrt')),   
)

area= alt.Chart(grupo_import_country.head(10)).mark_area(opacity=0.3).encode(
    x=alt.X('origin',sort=alt.SortArray(['Porcentaje_apoyo'])),
    y=alt.Y('Porcentaje_apoyo',stack=None, )
    # color="product"
).mark_text(align='left',dx=15,size=13).encode(text='total_value:Q')

(line + area ).properties(width=500, title="Historial de top 5 rutas den 2020")

base = alt.Chart(grupo_import_country.head(10)).encode(
    theta=alt.Theta("Porcentaje_apoyo:Q", stack=True),
    radius=alt.Radius("Porcentaje_apoyo", scale=alt.Scale(type="sqrt", zero=True, rangeMin=30)),
    color=alt.Color("Porcentaje_apoyo:N", legend=alt.Legend(title='% de rutas')),   
)

c1 = base.mark_arc(innerRadius=20, stroke="#ffF")

c2 = base.mark_text(radiusOffset=10 ,size=14).encode(text="origin:N")

(c1 + c2).properties(width=500, title="Porcentaje rutas de exportaciones por país")

    origin  register_id  Porcentaje_apoyo
0    China          615            16.859
1    Japan          445            12.198
2      USA          376            10.307
3  Germany          369            10.115
4   Mexico          366            10.033


alt.LayerChart(...)

#### Historico por pais top 5 importación

In [74]:
list_histo_3=[]
for index in range(5):
    first=left_join.iloc[index]
    query=f'origin == "{first.origin_x}" '
    first_route_hist = df_import.query(query).groupby(['year','origin']).count()['register_id']
    first_route_hist = first_route_hist.to_frame().reset_index()
    list_histo_3.append(first_route_hist)

#### Formula de prediccion

In [ ]:
# Regrecion lineal -->

#### Grafica Historico top 5 países mas relevantes

In [60]:
print(list_histo_3[0].columns.values)
line = alt.Chart(list_histo_3[0]).mark_line().encode(
    x=alt.X('year:O' , title='Año'),
    y=alt.Y('register_id', title='Numero de rutas'),
    color=alt.Color("origin")
)

line1 = alt.Chart(list_histo_3[1]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

line2 = alt.Chart(list_histo_3[2]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

line3 = alt.Chart(list_histo_3[3]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

line4 = alt.Chart(list_histo_3[4]).mark_line().encode(
    x=alt.X('year:O',title='Año'),
    y=alt.Y('register_id'),
    color=alt.Color("origin")
)

area= alt.Chart(list_histo_3[0]).mark_area(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("origin:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area1= alt.Chart(list_histo_3[1]).mark_area(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area2= alt.Chart(list_histo_3[2]).mark_area(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area3= alt.Chart(list_histo_3[3]).mark_area(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

area4= alt.Chart(list_histo_3[4]).mark_area(opacity=0.3).encode(
    x="year:O",
    y=alt.Y("register_id:Q", stack=None),
    color="origin"
).mark_text(align='center',dx=15,size=13).encode(text='register_id:Q')

(line + line1 + line2 + line3  + line4 + area + area1 + area2 + area3 + area4 ).properties(width=500, title="Historial de top 5 rutas den 2020")

# print(list_histo_3[1])

['year' 'origin' 'register_id']


alt.LayerChart(...)

In [23]:
df_import = testDataframe[(testDataframe.direction == "Imports")]
prediction = df_import.query('origin == "China"')
print(prediction.columns.values)
# prediction = df_import.groupby(['origin']).count()['register_id'].sort_values(ascending=False).to_frame().reset_index()
alt.Chart(prediction).mark_circle(size=60).encode(
    x='date:T',
    y='total_value',
    color='product',
    tooltip=['transport_mode', 'product', 'company_name', 'origin']
)#.interactive()


['register_id' 'direction' 'origin' 'destination' 'year' 'date' 'product'
 'transport_mode' 'company_name' 'total_value']


alt.Chart(...)